In [1]:
# Import libraries
import warnings
warnings.filterwarnings('ignore')

# For importing .xml files
import xml.etree.ElementTree as ET

# For handeling dataframes
import pandas as pd
import numpy as np
import re                                  # For regular expression operations
import string                              # For string operations
import math
import gc

# For text preprocessing
import nltk                                # Natural Language Toolkit
from nltk.corpus import stopwords          # For stop words that come with NLTK

# from nltk.stem import PorterStemmer as ps  # For stemming

# For building neural netwrok models
import sklearn
# from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import layers, losses
from tensorflow.keras.layers import Dense, Conv1D, Flatten, MaxPooling1D, Dropout, Bidirectional
from tensorflow.keras.optimizers import SGD, Adam

# For model evaluation
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import average_precision_score, precision_recall_curve
from sklearn.metrics import auc, plot_precision_recall_curve
from sklearn.metrics import roc_auc_score
import time
import random

# For text agumenters
import os
os.environ["MODEL_DIR"] = '../model'
import nlpaug
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
import nlpaug.flow as nafc
import nlpaug.model.word_stats as nmw

from nlpaug.util import Action
import tqdm
from tqdm import tqdm # To show the progress bar
from joblib import Parallel, delayed

nltk.download('averaged_perceptron_tagger')  # 
nltk.download('wordnet')                     # For Synonym augmenter
nltk.download('punkt')                       # For BackTranslation augmenter

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\rakovski\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\rakovski\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rakovski\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
# pip install torch torchvision

In [3]:
# pip install nlpaug

In [4]:
# import sys

In [5]:
labels_all = pd.read_csv("data/labels_all.csv")
labels_all.drop(['Unnamed: 0'], axis=1, inplace=True)

text_all = pd.read_csv('data/text_all.csv')
text_all.drop(['Unnamed: 0'], axis=1, inplace=True)

#  Create binary labels for id
disease_types = ['Hypertriglyceridemia','Venous Insufficiency','Asthma','Gout','OSA','PVD','Gallstones','OA','GERD',
                'Depression','Obesity','CHF','Hypercholesterolemia','CAD','Diabetes','Hypertension']

In [6]:
# Make the outcome variables (disease types) binary
def make_binary(labels_df):
    for i in range(16):
        # In the disease column, code the disease of interest as 1 and other disease types as 0 to indicate 
        # whether this row is about this particular disease
        labels_df["disease_"+str(i)] = [1 if x==disease_types[i] else 0 for x in labels_df["disease"]]
        # In the label column, code Y as 1 and everything else as 0 to indicate whether a disease is present
        # (it may or may not be about this particular disease)
        labels_df["label_"+str(i)] = [1 if x=="Y" else 0 for x in labels_df["label"]] 
        # Create a "This_Disease" column that indicates whether a patient has this particular disease or not
        # A patient is coded as having this disease only when (disease is 1) AND (label is 1)
        labels_df["This_Disease_"+str(i)] = labels_df["disease_"+str(i)] * labels_df["label_"+str(i)] 
        labels_df.drop(["disease_"+str(i), "label_"+str(i)], axis=1, inplace=True)

    # Now that all useful info in "disease" and "Label" are combined in column "This_Disease", no longer need disease and label
    labels_df.drop(['label', 'disease'], axis=1, inplace=True)
    # Duplicates after removing disease and label are due to 0's in the "disease" column if the patient had other disease info
    labels_df.drop_duplicates(inplace = True)
    # Aggregate the binary values into one row for each id
    labels_df = labels_df.groupby('id').aggregate('sum').reset_index()
    
    return(labels_df)

In [7]:
binary_labels = make_binary(labels_all)

# Calculate the disease prevalence for each disease type
prevalences = (binary_labels.aggregate('sum')/1237).iloc[1:,]

# Combine the text data and the labels
df = text_all.merge(binary_labels, on="id", how="left")

In [8]:
########################################################################
# NLP Preprocessing
########################################################################
# Download the stopwords from NLTK
nltk.download('stopwords')
stopwords_english = stopwords.words('english') 

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rakovski\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
# Clean text
def preprocessing(text):
    text = "".join([x for x in text if x not in string.punctuation]) # Remmove punctuations 
    text = ' '.join(['' if (len(x) <= 2) else x for x in text.split()]) # Remove short words
    text = "".join([x.lower() for x in text]) # Convert to lower case
    text = ' '.join(['' if (x in stopwords_english) else x for x in text.split()]) # Remove stopwords
    text = re.sub("(\W|\d+|\n)", " ", text).strip() # remove spaces, digits and line breaks
    return(text)

In [10]:
#######################################################################
# Functions
#######################################################################

# Define a function to evaluate the model performance in terms of F1 score
def evaluate(model, X, y):
#     pred = model.predict_classes(X)
    pred = (model.predict(X) > 0.5).astype("int32")[:,1]
    acc = np.sum(y == pred)/len(pred)
    
    tn, fp, fn, tp = confusion_matrix(y, pred).ravel()

    precision = tp/(tp + fp)
    recall = tp/(tp + fn)
    specificity = tn/(tn + fp)
    f1 = (2*precision*recall)/(precision + recall)
    
    y_pred = model.predict(X)
    auc_roc = round(roc_auc_score(y, y_pred[:,1]),4)
    pre, rec, thresholds = precision_recall_curve(y, y_pred[:,1])
    auc_pr = round(auc(rec, pre),4)
    return(auc_roc, auc_pr, acc, precision, recall, specificity, f1)

In [11]:
def _tokenizer(text, token_pattern=r"(?u)\b\w\w+\b"):
    token_pattern = re.compile(token_pattern)
    return token_pattern.findall(text)

In [12]:
# Arguments for word augmenter
aug_min_2=100
aug_max_2=1000
top_k=500 
aug_p=0.3

# Define the augmenters
# Character Augmenters
augs = []

# TFIDF Augmenter
augs.append(naw.TfIdfAug(model_path='.', tokenizer=_tokenizer, action='insert',
                              aug_min=aug_min_2,aug_max=aug_max_2, aug_p=aug_p, top_k=top_k))
# TFIDF Augmenter
augs.append(naw.TfIdfAug(model_path='.', tokenizer=_tokenizer, action='substitute',
                              aug_min=aug_min_2,aug_max=aug_max_2, aug_p=aug_p, top_k=top_k))

In [13]:
# Create the Transformer model
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, max_length, vocab_size, embedding_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim)
        self.pos_emb = layers.Embedding(input_dim=max_length, output_dim=embedding_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=max_length, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions
    
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, dropout_rate=0.3):
        super(TransformerBlock, self).__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embedding_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embedding_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(dropout_rate)
        self.dropout2 = layers.Dropout(dropout_rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)
    

def transformer_model(vocab_size, embedding_dim,max_length,dropout_rate):   
    inputs = layers.Input(shape=(max_length,))
    embedding_layer = TokenAndPositionEmbedding(max_length, vocab_size, embedding_dim)
    x = embedding_layer(inputs)
    transformer_block = TransformerBlock(embedding_dim, num_heads, ff_dim)
    x = transformer_block(x)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dropout(dropout_rate)(x)
    outputs = layers.Dense(2, activation="sigmoid")(x)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    opt = tf.keras.optimizers.Adam()
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['AUC'])
    
    return(model)

In [14]:
#######################################################################
# Modeling
#######################################################################

# Set the number of epochs and the batch size
epochs = 20 ###################################################################
batch_size = 32
# tokenizer = Tokenizer(num_words=5000) # get the frequency of all tokens and use the 5000 most common ones
max_length = 557 # assumes the first 522 words are the most important, and make all sequences with length 525
dropout_rate = 0.3
embedding_dim = 200
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer

time_1 = time.time()

X = df['text']

for i in range(16):
    # Dummify the label variable event
    event_categorical = to_categorical(df['This_Disease_'+str(i)])
    prevalence = prevalences.iloc[i]
    dis_type = disease_types[i]
    j = 0 # to keep track of the iteration number
    time_start = time.time()
    
    # Run the model 10 times with a different split each time  
    sss = StratifiedShuffleSplit(n_splits=10, test_size=0.25, random_state=0)
    for train_index, test_index in sss.split(X, event_categorical):
        time_s = time.time()
        
        j += 1
        iteration = "iter" + str(j)
        x_train, x_test = X[train_index], X[test_index]
        y_train, y_test = event_categorical[train_index], event_categorical[test_index]
        
          
        
        # Select the samples to be augmented 
        if prevalence < 0.5: # If disease presence is the minority
            x_aug_0 = x_train[y_train[:,1]==1] # Augment disease presence cases
            y_aug_0 = y_train[y_train[:,1]==1,0]

        else: # If disease absence is the minority
            x_aug_0 = x_train[y_train[:,0]==1] # Augment disease absence cases
            y_aug_0 = y_train[y_train[:,0]==1,0]
            
        
        x_aug_0  = [re.sub("(\d+|\n)", " ", elem).strip() for elem in x_aug_0]
        x_aug_0 = [str(x).lower() for x in x_aug_0]

        # Tokenize input
        train_x_tokens = [_tokenizer(x) for x in x_aug_0]

        # Train TF-IDF model
        tfidf_model = nmw.TfIdf()
        tfidf_model.train(train_x_tokens)
        tfidf_model.save('.')    

        x_aug_0 = np.array(x_aug_0)

        # Randomly augment a predetermined number of the selected cases
        length = len(x_aug_0) # Total number of texts availble for aumentation
        num = math.ceil(abs(prevalence - 0.5)*len(x_train))*2 # Number of samples needed to balance the classes in the training set
        if num <= length:
            # If the number of texts to be augmented is larger than the number of samples needed
            samples = np.random.choice(range(length), size=num, replace=False) # Generate random indices without replacement
        else:
            samples = np.random.choice(range(length), size=num, replace=True) # Generate random indices with replacement
               

        for k in range(2): # Go through each TFIDF methods
            DA_start = time.time()
            aug = augs[k]
            
            new_text = []
            new_y_0 = []
            new_y_1 = []
            
            for ii in samples:   
                text = x_aug_0[ii]
                label_aug = y_aug_0[ii]
                augmented_text = aug.augment(text)
                    
                new_text.append(augmented_text)
                new_y_0.append(label_aug) # Assign the label of the original text to the augmented text
                new_y_1.append(1-label_aug) # Assign the label of the original text to the augmented text

            # Append the augmented texts and their corresponding labels to the original data
            x_aug = np.append(x_train, new_text)
            y_0 = np.append(y_train[:,0], new_y_0)
            y_1 = np.append(y_train[:,1], new_y_1)
            y_aug = (np.vstack((y_0,y_1))).T
            #################################################################################

            # Shuffle the order of the augmented dataset
            augmented = pd.DataFrame((np.vstack((x_aug, y_aug[:,0], y_aug[:,1]))).T)
            augmented = augmented.sample(frac=1).reset_index(drop=True)

            DA_time = time.time() - DA_start
            augmenter = "aug_TFIDF_"+str(k)
            with open('other/DA_until_balanced_TFIDF_Transformer.csv','a') as fd:
                fd.write(f'{dis_type},{iteration},{augmenter},DA Running Time:,{DA_time}\n')

            # Convert the dataframe to numpy arrays
            x_train_1 = np.array(augmented.iloc[:,0])
            y_train_1 = np.array(augmented.iloc[:,[1,2]])
            y_train_1 = y_train_1.astype('float32')

            # Run CNN model
            cnn_time_start = time.time()

            # Preprocess the text
            vfunc = np.vectorize(preprocessing)
            x_train_1 = vfunc(x_train_1)
            x_test_1 = vfunc(x_test)

            # Tokenize the text   
            tokenizer = Tokenizer(num_words=5000) # get the frequency of all tokens and use the 5000 most common ones
            tokenizer.fit_on_texts(x_train_1)
            x_train_1 = tokenizer.texts_to_sequences(x_train_1)
            x_test_1 = tokenizer.texts_to_sequences(x_test_1)
            vocab_size = len(tokenizer.word_index) + 1 # plus the reserved index 0
            word_index = tokenizer.word_index

            # Pad the sequences with 0's
            x_train_1 = pad_sequences(x_train_1, padding='post', maxlen=max_length) 
            x_test_1 = pad_sequences(x_test_1, padding='post', maxlen=max_length)

            # Fit the Transformer model
            mymodel = transformer_model(vocab_size, embedding_dim,max_length,dropout_rate)
            mymodel.fit(x_train_1, y_train_1, epochs=epochs, batch_size=batch_size)

            # Collect and log evaluation metrics
            auc_roc, auc_pr, acc, precision, recall, specificity, f1 = evaluate(mymodel, x_test_1, y_test[:,1])
            cnn_time = time.time() - cnn_time_start

            with open('other/DA_until_balanced_TFIDF_Transformer.csv','a') as fd:
                fd.write(f'{dis_type},{augmenter},{auc_roc},{auc_pr},{acc},{precision},{recall},{specificity},{f1},{cnn_time},{len(x_aug)}\n')

            del tokenizer, mymodel, vocab_size, word_index, x_aug, y_aug, augmented, x_train_1, x_test_1, y_train_1
            gc.collect()
            
    running_time = time.time() - time_start
    with open('other/DA_until_balanced_TFIDF_Transformer.csv','a') as fd:
            fd.write(f'{dis_type},10 iteration running time,{running_time}\n')

Epoch 1/20
55/55 [==============================] - 48s 849ms/step - loss: 0.2075 - auc: 0.9729
Epoch 2/20
55/55 [==============================] - 47s 852ms/step - loss: 0.0274 - auc: 0.9987
Epoch 3/20
55/55 [==============================] - 47s 853ms/step - loss: 0.0125 - auc: 0.9996
Epoch 4/20
55/55 [==============================] - 47s 857ms/step - loss: 0.0184 - auc: 0.9998
Epoch 5/20
55/55 [==============================] - 47s 854ms/step - loss: 0.0028 - auc: 1.0000
Epoch 6/20
55/55 [==============================] - 47s 850ms/step - loss: 0.0019 - auc: 1.0000
Epoch 7/20
55/55 [==============================] - 47s 859ms/step - loss: 1.2696e-04 - auc: 1.0000
Epoch 8/20
55/55 [==============================] - 47s 854ms/step - loss: 8.4099e-05 - auc: 1.0000
Epoch 9/20
55/55 [==============================] - 47s 851ms/step - loss: 7.0358e-05 - auc: 1.0000
Epoch 10/20
55/55 [==============================] - 47s 850ms/step - loss: 5.9315e-05 - auc: 1.0000
Epoch 11/20
55/55 [====

55/55 [==============================] - 51s 928ms/step - loss: 0.0070 - auc: 1.0000
Epoch 4/20
55/55 [==============================] - 51s 930ms/step - loss: 0.0022 - auc: 1.0000
Epoch 5/20
55/55 [==============================] - 51s 921ms/step - loss: 8.5395e-04 - auc: 1.0000
Epoch 6/20
55/55 [==============================] - 50s 918ms/step - loss: 1.4526e-04 - auc: 1.0000
Epoch 7/20
55/55 [==============================] - 51s 930ms/step - loss: 9.7484e-05 - auc: 1.0000
Epoch 8/20
55/55 [==============================] - 51s 923ms/step - loss: 6.7318e-05 - auc: 1.0000
Epoch 9/20
55/55 [==============================] - 51s 927ms/step - loss: 6.6897e-05 - auc: 1.0000
Epoch 10/20
55/55 [==============================] - 51s 922ms/step - loss: 5.3806e-05 - auc: 1.0000
Epoch 11/20
55/55 [==============================] - 51s 927ms/step - loss: 4.9717e-05 - auc: 1.0000
Epoch 12/20
55/55 [==============================] - 51s 928ms/step - loss: 4.5286e-05 - auc: 1.0000
Epoch 13/20
55/5

55/55 [==============================] - 54s 990ms/step - loss: 2.7509e-04 - auc: 1.0000
Epoch 7/20
55/55 [==============================] - 54s 990ms/step - loss: 1.9508e-04 - auc: 1.0000
Epoch 8/20
55/55 [==============================] - 56s 1s/step - loss: 1.5060e-04 - auc: 1.0000
Epoch 9/20
55/55 [==============================] - 54s 980ms/step - loss: 1.2204e-04 - auc: 1.0000
Epoch 10/20
55/55 [==============================] - 54s 976ms/step - loss: 9.3725e-05 - auc: 1.0000
Epoch 11/20
55/55 [==============================] - 54s 975ms/step - loss: 9.0603e-05 - auc: 1.0000
Epoch 12/20
55/55 [==============================] - 54s 979ms/step - loss: 7.0749e-05 - auc: 1.0000
Epoch 13/20
55/55 [==============================] - 54s 975ms/step - loss: 6.6949e-05 - auc: 1.0000
Epoch 14/20
55/55 [==============================] - 54s 975ms/step - loss: 6.0035e-05 - auc: 1.0000
Epoch 15/20
55/55 [==============================] - 54s 975ms/step - loss: 5.2476e-05 - auc: 1.0000
Epoch 16

55/55 [==============================] - 56s 1s/step - loss: 4.6241e-05 - auc: 1.0000
Epoch 10/20
55/55 [==============================] - 56s 1s/step - loss: 4.3916e-05 - auc: 1.0000
Epoch 11/20
55/55 [==============================] - 56s 1s/step - loss: 3.6204e-05 - auc: 1.0000
Epoch 12/20
55/55 [==============================] - 56s 1s/step - loss: 3.4332e-05 - auc: 1.0000
Epoch 13/20
55/55 [==============================] - 56s 1s/step - loss: 3.0081e-05 - auc: 1.0000
Epoch 14/20
55/55 [==============================] - 55s 1s/step - loss: 2.8129e-05 - auc: 1.0000
Epoch 15/20
55/55 [==============================] - 55s 1s/step - loss: 2.5530e-05 - auc: 1.0000
Epoch 16/20
55/55 [==============================] - 55s 1s/step - loss: 2.4789e-05 - auc: 1.0000
Epoch 17/20
55/55 [==============================] - 55s 1s/step - loss: 2.3113e-05 - auc: 1.0000
Epoch 18/20
55/55 [==============================] - 55s 1s/step - loss: 2.1025e-05 - auc: 1.0000
Epoch 19/20
55/55 [=============

55/55 [==============================] - 63s 1s/step - loss: 5.3676e-05 - auc: 1.0000
Epoch 14/20
55/55 [==============================] - 62s 1s/step - loss: 4.6288e-05 - auc: 1.0000
Epoch 15/20
55/55 [==============================] - 62s 1s/step - loss: 4.1095e-05 - auc: 1.0000
Epoch 16/20
55/55 [==============================] - 62s 1s/step - loss: 3.9400e-05 - auc: 1.0000
Epoch 17/20
55/55 [==============================] - 64s 1s/step - loss: 3.4671e-05 - auc: 1.0000
Epoch 18/20
55/55 [==============================] - 64s 1s/step - loss: 3.0800e-05 - auc: 1.0000
Epoch 19/20
55/55 [==============================] - 64s 1s/step - loss: 2.8154e-05 - auc: 1.0000
Epoch 20/20
55/55 [==============================] - 63s 1s/step - loss: 2.7896e-05 - auc: 1.0000
Epoch 1/20
55/55 [==============================] - 72s 1s/step - loss: 0.2856 - auc: 0.9499
Epoch 2/20
55/55 [==============================] - 71s 1s/step - loss: 0.0183 - auc: 0.9990
Epoch 3/20
55/55 [========================

...

43/43 [==============================] - 63s 1s/step - loss: 0.0311 - auc: 0.9993
Epoch 6/20
43/43 [==============================] - 63s 1s/step - loss: 0.0153 - auc: 0.9999
Epoch 7/20
43/43 [==============================] - 63s 1s/step - loss: 0.0033 - auc: 1.0000
Epoch 8/20
43/43 [==============================] - 63s 1s/step - loss: 6.1694e-04 - auc: 1.0000
Epoch 9/20
43/43 [==============================] - 63s 1s/step - loss: 4.0564e-04 - auc: 1.0000
Epoch 10/20
43/43 [==============================] - 63s 1s/step - loss: 3.2544e-04 - auc: 1.0000
Epoch 11/20
43/43 [==============================] - 63s 1s/step - loss: 2.4718e-04 - auc: 1.0000
Epoch 12/20
43/43 [==============================] - 63s 1s/step - loss: 2.0178e-04 - auc: 1.0000
Epoch 13/20
43/43 [==============================] - 63s 1s/step - loss: 1.6321e-04 - auc: 1.0000
Epoch 14/20
43/43 [==============================] - 63s 1s/step - loss: 1.5700e-04 - auc: 1.0000
Epoch 15/20
43/43 [=============================

43/43 [==============================] - 63s 1s/step - loss: 3.1194e-04 - auc: 1.0000
Epoch 11/20
43/43 [==============================] - 63s 1s/step - loss: 2.8459e-04 - auc: 1.0000
Epoch 12/20
43/43 [==============================] - 64s 1s/step - loss: 2.2012e-04 - auc: 1.0000
Epoch 13/20
43/43 [==============================] - 64s 1s/step - loss: 1.9863e-04 - auc: 1.0000
Epoch 14/20
43/43 [==============================] - 64s 1s/step - loss: 1.5984e-04 - auc: 1.0000
Epoch 15/20
43/43 [==============================] - 63s 1s/step - loss: 1.5067e-04 - auc: 1.0000
Epoch 16/20
43/43 [==============================] - 63s 1s/step - loss: 1.3343e-04 - auc: 1.0000
Epoch 17/20
43/43 [==============================] - 63s 1s/step - loss: 1.2335e-04 - auc: 1.0000
Epoch 18/20
43/43 [==============================] - 63s 1s/step - loss: 1.1103e-04 - auc: 1.0000
Epoch 19/20
43/43 [==============================] - 63s 1s/step - loss: 1.0009e-04 - auc: 1.0000
Epoch 20/20
43/43 [=============

43/43 [==============================] - 64s 1s/step - loss: 1.5523e-04 - auc: 1.0000
Epoch 16/20
43/43 [==============================] - 64s 1s/step - loss: 1.3920e-04 - auc: 1.0000
Epoch 17/20
43/43 [==============================] - 63s 1s/step - loss: 1.1728e-04 - auc: 1.0000
Epoch 18/20
43/43 [==============================] - 64s 1s/step - loss: 1.1458e-04 - auc: 1.0000
Epoch 19/20
43/43 [==============================] - 64s 1s/step - loss: 1.0294e-04 - auc: 1.0000
Epoch 20/20
43/43 [==============================] - 63s 1s/step - loss: 9.4618e-05 - auc: 1.0000
Epoch 1/20
43/43 [==============================] - 65s 1s/step - loss: 0.6489 - auc: 0.6719
Epoch 2/20
43/43 [==============================] - 64s 1s/step - loss: 0.3498 - auc: 0.9252
Epoch 3/20
43/43 [==============================] - 64s 1s/step - loss: 0.1974 - auc: 0.9766
Epoch 4/20
43/43 [==============================] - 64s 1s/step - loss: 0.0916 - auc: 0.9949
Epoch 5/20
43/43 [==============================] - 

43/43 [==============================] - 64s 1s/step - loss: 9.2751e-05 - auc: 1.0000
Epoch 1/20
43/43 [==============================] - 65s 1s/step - loss: 0.6371 - auc: 0.6913
Epoch 2/20
43/43 [==============================] - 64s 1s/step - loss: 0.3393 - auc: 0.9287
Epoch 3/20
43/43 [==============================] - 63s 1s/step - loss: 0.1462 - auc: 0.9882
Epoch 4/20
43/43 [==============================] - 63s 1s/step - loss: 0.0607 - auc: 0.9978
Epoch 5/20
43/43 [==============================] - 63s 1s/step - loss: 0.0361 - auc: 0.9992
Epoch 6/20
43/43 [==============================] - 63s 1s/step - loss: 0.0186 - auc: 0.9998
Epoch 7/20
43/43 [==============================] - 63s 1s/step - loss: 0.0086 - auc: 0.9996
Epoch 8/20
43/43 [==============================] - 63s 1s/step - loss: 0.1207 - auc: 0.9911
Epoch 9/20
43/43 [==============================] - 63s 1s/step - loss: 0.0411 - auc: 0.9991
Epoch 10/20
43/43 [==============================] - 63s 1s/step - loss: 0.00

Epoch 6/20
43/43 [==============================] - 67s 2s/step - loss: 0.0160 - auc: 0.9999
Epoch 7/20
43/43 [==============================] - 67s 2s/step - loss: 0.0016 - auc: 1.0000
Epoch 8/20
43/43 [==============================] - 66s 2s/step - loss: 5.7401e-04 - auc: 1.0000
Epoch 9/20
43/43 [==============================] - 67s 2s/step - loss: 3.9996e-04 - auc: 1.0000
Epoch 10/20
43/43 [==============================] - 67s 2s/step - loss: 3.3045e-04 - auc: 1.0000
Epoch 11/20
43/43 [==============================] - 66s 2s/step - loss: 2.5843e-04 - auc: 1.0000
Epoch 12/20
43/43 [==============================] - 66s 2s/step - loss: 2.1775e-04 - auc: 1.0000
Epoch 13/20
43/43 [==============================] - 66s 2s/step - loss: 1.8044e-04 - auc: 1.0000
Epoch 14/20
43/43 [==============================] - 66s 2s/step - loss: 1.7075e-04 - auc: 1.0000
Epoch 15/20
43/43 [==============================] - 67s 2s/step - loss: 1.5224e-04 - auc: 1.0000
Epoch 16/20
43/43 [=============

## Parallel Computing

In [15]:
# pip install dill

In [16]:
# import dill as pickle

In [17]:
# import nltk
# nltk.download('omw')

In [18]:
# Parallelizing the computation for all 16 disease conditions

# Parallel(n_jobs=16,backend="threading")(delayed(main_cnn)(x) for x in range(16))
# Parallel(n_jobs=-2,backend="threading")(delayed(main_cnn)(x) for x in [13])

In [19]:
time_2 = time.time() - time_1

with open('other/DA_until_balanced_TFIDF_Transformer.csv','a') as fd:
            fd.write(f'Total running time, {time_2}\n')